#  IAPWS-IF97 Libraries

## 1 Introduction to IAPWS-IF97

http://www.iapws.org/relguide/IF97-Rev.html

This formulation is recommended for industrial use (primarily the steam power industry) for the calculation of thermodynamic properties of ordinary water in its fluid phases, including vapor-liquid equilibrium. 

http://www.iapws.org/relguide/IF97-Rev.pdf

The release also contains "backward" equations to allow calculations with certain common sets of independent variables to be made without iteration; these equations may also be used to provide good initial guesses for iterative solutions. 
Since the release was first issued, it has been supplemented by several additional "backward" equations that are available for use if desired; these are for p(h,s) in Regions 1 and 2, T(p,h), v(p,h), T(p,s), v(p,s) in Region 3, p(h,s) in Region 3 with auxiliary equations for independent variables h and s, and v(p,T) in Region 3. 

![if97](./img/If97.jpg)

## 2 Python library for IAPWS 

  https://github.com/jjgomera/iapws
    
Numpy,scipy: library with mathematic and scientific tools  

In [ ]:
!pip install iapws

In [ ]:
from iapws import IAPWS97
sat_steam=IAPWS97(P=1,x=1)                #saturated steam with known P
sat_liquid=IAPWS97(T=370, x=0)            #saturated liquid with known T
steam=IAPWS97(P=2.5, T=500)               #steam with known P and T
print(sat_steam.h, sat_liquid.h, steam.h) #calculated enthalpies

## 3 SEUIF97 shared Library(Educational Edition:Windows) 

https://github.com/PySEE/EDUIF97

   The high-speed library is written in ANSI C,through the  library, the results of the IAPWS-IF97  are accurately produced at above 3 times computational speed.
   
>         Algorithm of the High-speed IAPWS-IF97 Library: 
>
>        王培红,贾俊颖,程懋华. 水和水蒸汽热力性质IAPWS-IF97公式的通用计算模型[J]. 动力工程. 2001 21(6)：1564-1567(EI)
 
The shared library  is suitable for developers to calculate the properties of water and steam  in extensive process simulations, such as Computational Fluid Dynamics (CFD), heat cycle calculations, simulations of non-stationary processes, and real-time process optimizations, where the direct IAPWS-IF97 implementation may be unsuitable because of their computing time consumption.
 
For Windows users, convenient binary packages are provided.
 
         IF97 shared library: Windows(32/64):libseuif97.dll
 
The package includes bindings for Python
        
        Python API：seuif97.py
        
### 3.1  Python 

* Python API: **seuif97.py** 

#### Windows32/64
  
1. copy **libseuif97.dll**  to a default path of Windows dll
      
        C:\Windows\system
   
2. copy **seuif97.py** to a default path of Python lib
    
        C:\Python35\Lib  


### 3.2 SEUIF97 share Library（API)

### Functions 

```c
   seupt( pressure,    temperature, propertyID)
   seuph( pressure,    enthalpy,    propertyID)
   seups( pressure,    entropy,     propertyID)
   seuhs( enthalpy,    entropy,     propertyID)
   seupx( pressure,    quality,     propertyID)
   seutx( temperature, quality,     propertyID)
```

   *  propertyID  - int，0-29,(see **Properties in libseuif97**)

### Function for process  
   
*  1 Isentropic Enthalpy Drop： ishd( pi,ti,pe)
    
        pi - double,inlet P; ti - double,inlet T

        pe - double,outlet P

* 2 Isentropic Efficiency： ief( pi,ti,pe,te)

         pi  -double，inlet P; ti  - double,inlet T

         pe  -double，outlet P; te  - double, outlet T

### 3.2.1 Python 接口：

seuif97.py

In [ ]:
# -*- coding: utf-8 -*-

from ctypes import *
from platform import *

flib = windll.LoadLibrary(cdll_names[osplat])
prototype = WINFUNCTYPE(c_double, c_double, c_double, c_int)

# ---(p,t) ----------------

def pt(p, t, pid):
    f = prototype(("seupt", flib),)
    result = f(p, t, pid)
    return result


def pt2h(p, t):
    f = prototype(("seupt", flib),)
    result = f(p, t, 4)
    return result

#### 3.2.1.1 Example1

In [13]:
from seuif97 import *
p=16.12
t=562.1

h=pt2h(p,t)
s=pt2s(p,t)
print('h:',h,'s: ',s)

t1=ph(p,h,1)
t2=ps(p,s,1)

print('t1: ',t1,' t2 ',t2)

p1=hs(h,s,0)
t3=hs(h,s,1)
print('p1: ',p1,' t3 ',t3)


h: 3471.7848838836876 s:  6.518652433515181
t1:  562.1000000000004  t2  562.1000000000001
p1:  16.120000000000005  t3  562.1000000000001


#### 3.2.2.2 Example2: Thermodynamic Process  

In [12]:
from seuif97 import *

p1=16.1
t1=535.2
p2=3.56
t2=315.1

hdis=ishd(p1,t1,p2) # 等熵焓降
ef=ief(p1,t1,p2,t2) # 过程效率

print('等熵焓降 =',hdis,'kJ/kg')
print('过程效率 = %.2f%%'%ef)

等熵焓降 = 426.2965108499634 kJ/kg
过程效率 = 89.46%


#### 3.2.2.3 T-s Diagram

In [ ]:
%matplotlib inline
"""
T-s Diagram

1 isoenthalpic lines isoh(200, 3600)kJ/kg
2 isobar lines isop(611.657e-6,100)MPa
3 saturation lines x=0,x=1
4 isoquality lines x(0.1,0.9)

"""
from seuif97 import pt2h, ph2t, ph2s, tx2s
import numpy as np
import matplotlib.pyplot as plt

Pt=611.657e-6
Tc=647.096

xAxis = "s"
yAxis = "T"
title = {"T": "T, ºC", "s": "s, kJ/kgK"}
plt.title("%s-%s Diagram" % (yAxis, xAxis))
plt.xlabel(title[xAxis])
plt.ylabel(title[yAxis])
plt.xlim(0, 11.5)
plt.ylim(0, 800)
plt.grid()

isoh = np.linspace(200, 3600, 18)
isop = np.array([Pt,0.001,0.002,0.004,0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1.0,
                 2.0, 5.0, 10.0, 20.0, 50.0, 100.0])
for h in isoh:
    T = np.array([ph2t(p, h) for p in isop])
    S = np.array([ph2s(p, h) for p in isop])
    plt.plot(S, T, 'b', lw=0.5)

for p in isop:
    T = np.array([ph2t(p, h) for h in isoh])
    S = np.array([ph2s(p, h) for h in isoh])
    plt.plot(S, T, 'b', lw=0.5)

tc = Tc - 273.15
T = np.linspace(0.01, tc, 100)
for x in np.array([0, 1.0]):
    S = np.array([tx2s(t, x) for t in T])
    plt.plot(S, T, 'r', lw=1.0)

for x in np.linspace(0.1, 0.9, 11):
    S = np.array([tx2s(t, x) for t in T])
    plt.plot(S, T, 'r--', lw=0.5)

plt.show()

#### 3.2.2.4 H-S Diagram

In [ ]:
%matplotlib inline
"""
h-s Diagram

1 Calculating Isotherm lines isot(0.0,800)ºC
2 Calculating Isobar lines  isop(611.657e-6, 100)Mpa
3 Calculating saturation lines x=0,x=1
4 Calculating isoquality lines x(0.1,0.9)
"""
from  seuif97 import pt2h,pt2s,tx2s,tx2h
import numpy as np
import matplotlib.pyplot as plt

xAxis = "s"
yAxis = "h"
title = { "h": "h, kJ/kg", "s": "s, kJ/kgK"}

plt.title("%s-%s Diagram" % (yAxis, xAxis))
plt.xlabel(title[xAxis])
plt.ylabel(title[yAxis])
plt.xlim(0, 12.2)
plt.ylim(0, 4300)
plt.grid()

Pt=611.657e-6

isot = np.array([0, 50, 100, 200, 300, 400, 500, 600, 700, 800])
isop = np.array([Pt,0.001, 0.01, 0.1, 1, 10, 20, 50, 100])
# Isotherm lines in ºC
for t in isot:
    h = np.array([pt2h(p,t) for p in isop])
    s = np.array([pt2s(p,t) for p in isop])
    plt.plot(s,h,'g',lw=0.5)

# Isobar lines in Mpa
for p in isop:
    h = np.array([pt2h(p,t) for t in isot])
    s = np.array([pt2s(p,t) for t in isot])
    plt.plot(s,h,'b',lw=0.5)

tc=647.096-273.15
T = np.linspace(0.1,tc,100)  
# saturation lines
for x in np.array([0,1.0]):
    h = np.array([tx2h(t,x) for t in T])
    s = np.array([tx2s(t,x) for t in T])
    plt.plot(s,h,'r',lw=1.0)

# Isoquality lines
isox=np.linspace(0.1,0.9,11)
for x in isox:
    h = np.array([tx2h(t,x) for t in T])
    s = np.array([tx2s(t,x) for t in T])
    plt.plot(s,h,'r--',lw=0.5)

plt.show()